In [133]:
import pandas as pd
import csv
import os
from statistics import mean
from textblob import TextBlob
import numpy as np

In [134]:
df_dataset = pd.read_csv("../dataset/dataset.csv")

In [135]:
df_dataset

,id,baslik,sarki_sozu,sarki_turu,eski_turkce_kelime_orani,fiil_orani,sifat_orani,zarf_orani,baglac_orani,unlem_orani,soru_orani,bilinmeyen_orani
0,0,Acem Kızı,çırpınıp da şan ovaya çıkınca\r\neğlen şan ova...,halk,0.204545,0.07,0.05,0.09,0.02,0.00,0.05,0.00
1,1,Açma zülüflerin yellere karşı,açma zülüflerin yellere karşı\r\nsenin zülfün ...,halk,0.148936,0.15,0.00,0.00,0.00,0.00,0.07,0.02
2,2,Ah bir ataş ver cigaramı yakayım,ah bir ataş ver cigaramı yakayım\r\nsen salın ...,halk,0.176471,0.28,0.08,0.00,0.00,0.00,0.00,0.02
3,3,Sana olan duyguları bir bilebilsen (Ah le yar ...,sana olan duyguları bir bilebilsen\r\nanlayabi...,halk,0.241935,0.16,0.04,0.05,0.04,0.00,0.00,0.01
4,4,Ahu gözlerini sevdiğim dilber,ahu gözlerini sevdiğim dilber\r\nsana bir sözü...,halk,0.111111,0.25,0.13,0.02,0.00,0.00,0.02,0.04
...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1195,ez - BENIM HAYALLER,heute eau de parfum in der goyardbag\r\nkomme ...,rap,0.072848,0.10,0.05,0.05,0.03,0.00,0.01,0.27
1196,1196,Canbay & Wolker - Fersah,heyhat kafam göçtü diyardan\r\ndönmedi kimse l...,rap,0.023411,0.12,0.06,0.03,0.01,0.02,0.00,0.07
1197,1197,grogi - balerin,yo\r\ni̇zmir\r\ngrogi\r\nanıl piyancı\r\ni̇zmi...,rap,0.018617,0.16,0.10,0.04,0.02,0.01,0.00,0.15
1198,1198,sansar salvo ağır roman,sansar salvo yerinde duramayan adam\r\nbi çok ...,rap,0.065972,0.15,0.14,0.03,0.05,0.00,0.00,0.05


In [136]:
df_rap=df_dataset.loc[df_dataset["sarki_turu"]=='rap']

In [137]:
df_rap=df_dataset.loc[df_dataset["sarki_turu"]=='rap']

sarki_adi=[]
lengths=[]
word_lengths=[]
repeated_word=[]
emtion_analysis=[] 
rap_sarki_sozleri=[]
for index,row in df_rap.iterrows():
    sarki_adi.append(row['baslik'])
    rap_sarki_sozleri.append(row['sarki_sozu'])
    #row="".join(row)
   
    lyrics = row["sarki_sozu"]
    lengths.append(len(lyrics.split(" ")))
    temp=[]
    for word in lyrics.split(" "):
        temp.append(len(word))

    turkish_text = row["sarki_sozu"]
    blob = TextBlob(turkish_text)
    polarity = blob.sentiment.polarity

    if polarity > 0:
        if polarity >= 0.5:
           # print("Mutlu")
            emtion_analysis.append(5)
        else:
            #print("Şaşırmış")
            emtion_analysis.append(2)
    elif polarity < 0:
        if polarity <= -0.5:
            #print("Kızgın")
            emtion_analysis.append(3)
        else:
            #print("Üzgün")
            emtion_analysis.append(4)
    else:
        
        if "korku" in turkish_text.lower() or "korkmak" in turkish_text.lower():
            if polarity <= -0.3:
                #print("Korkmuş")
                emtion_analysis.append(1)
            else:
                #print("Nötr")
                emtion_analysis.append(0)
        else:
            #print("Nötr")
            emtion_analysis.append(0)
    word_lengths.append(mean(temp))

    counts = {}
    str_row=lyrics.split(("\n"))
    temp_List2=[]
    for sentence in str_row:
        for word in sentence.split(" "):
            #print(word)
            #print()

            if word not in counts:
                counts[word] = 0
            counts[word] += 1
    for item in counts.values():
        temp_List2.append(item)
        #print(item)
        #print()
    #sorted_c=sorted(counts)
    repeated_word.append(sum(temp_List2)/len(temp_List2))

rap_plot_data=[mean(lengths),mean(word_lengths),mean(repeated_word)]
data = {'Şarki Adı':sarki_adi,
        'Toplam Kelime Sayısı': lengths,
        'Ortalama Kelime Uzunlukları': word_lengths,
        'Ortalama Kelime Tekrarı ':repeated_word,
        'Duygu Analizi':emtion_analysis}

rap_df = pd.DataFrame(data)
rap_df["eski_turkce_kelime_orani"] = df_rap["eski_turkce_kelime_orani"].values
rap_df["fiil_orani"] = df_rap["fiil_orani"].values
rap_df["sifat_orani"] = df_rap["sifat_orani"].values
rap_df["zarf_orani"] = df_rap["zarf_orani"].values
rap_df["baglac_orani"] = df_rap["baglac_orani"].values
rap_df["unlem_orani"] = df_rap["unlem_orani"].values
rap_df["soru_orani"] = df_rap["soru_orani"].values
rap_df["bilinmeyen_orani"] = df_rap["bilinmeyen_orani"].values
rap_df['Label']=0

In [138]:
rap_df

,Şarki Adı,Toplam Kelime Sayısı,Ortalama Kelime Uzunlukları,Ortalama Kelime Tekrarı,Duygu Analizi,eski_turkce_kelime_orani,fiil_orani,sifat_orani,zarf_orani,baglac_orani,unlem_orani,soru_orani,bilinmeyen_orani,Label
0,Sagopa Kajmer - Istakoz,346,6.624277,1.572491,0,0.070922,0.17,0.09,0.05,0.05,0.01,0.00,0.03,0
1,Ezhel - Felaket,120,8.166667,1.387931,0,0.118012,0.19,0.06,0.08,0.02,0.00,0.00,0.03,0
2,Sansar Salvo - Bela,125,7.400000,1.424779,3,0.055901,0.25,0.08,0.04,0.03,0.00,0.01,0.06,0
3,Norm Ender - Mekanın Sahibi,241,7.012448,1.464115,2,0.013072,0.16,0.08,0.04,0.02,0.07,0.00,0.11,0
4,Ben Fero - Mahallemiz Esmer,266,6.796992,1.429864,2,0.060127,0.13,0.07,0.04,0.01,0.01,0.01,0.12,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,ez - BENIM HAYALLER,248,6.217742,1.589474,2,0.072848,0.10,0.05,0.05,0.03,0.00,0.01,0.27,0
196,Canbay & Wolker - Fersah,239,7.380753,1.661111,0,0.023411,0.12,0.06,0.03,0.01,0.02,0.00,0.07,0
197,grogi - balerin,310,6.587097,1.928205,0,0.018617,0.16,0.10,0.04,0.02,0.01,0.00,0.15,0
198,sansar salvo ağır roman,240,7.158333,1.500000,0,0.065972,0.15,0.14,0.03,0.05,0.00,0.00,0.05,0


In [139]:
df_pop=df_dataset.loc[df_dataset["sarki_turu"]=='pop']

sarki_adi=[]
lengths=[]
word_lengths=[]
repeated_word=[]
emtion_analysis=[] 
pop_sarki_sozleri=[]
for index,row in df_pop.iterrows():
    sarki_adi.append(row['baslik'])
    pop_sarki_sozleri.append(row['sarki_sozu'])
    #row="".join(row)
   
    lyrics = row["sarki_sozu"]
    lengths.append(len(lyrics.split(" ")))
    temp=[]
    for word in lyrics.split(" "):
        temp.append(len(word))

    turkish_text = row["sarki_sozu"]
    blob = TextBlob(turkish_text)
    polarity = blob.sentiment.polarity

    if polarity > 0:
        if polarity >= 0.5:
           # print("Mutlu")
            emtion_analysis.append(5)
        else:
            #print("Şaşırmış")
            emtion_analysis.append(2)
    elif polarity < 0:
        if polarity <= -0.5:
            #print("Kızgın")
            emtion_analysis.append(3)
        else:
            #print("Üzgün")
            emtion_analysis.append(4)
    else:
        
        if "korku" in turkish_text.lower() or "korkmak" in turkish_text.lower():
            if polarity <= -0.3:
                #print("Korkmuş")
                emtion_analysis.append(1)
            else:
                #print("Nötr")
                emtion_analysis.append(0)
        else:
            #print("Nötr")
            emtion_analysis.append(0)
    word_lengths.append(mean(temp))

    counts = {}
    str_row=lyrics.split(("\n"))
    temp_List2=[]
    for sentence in str_row:
        for word in sentence.split(" "):
            #print(word)
            #print()

            if word not in counts:
                counts[word] = 0
            counts[word] += 1
    for item in counts.values():
        temp_List2.append(item)
        #print(item)
        #print()
    #sorted_c=sorted(counts)
    repeated_word.append(sum(temp_List2)/len(temp_List2))

pop_plot_data=[mean(lengths),mean(word_lengths),mean(repeated_word)]
data = {'Şarki Adı':sarki_adi,
        'Toplam Kelime Sayısı': lengths,
        'Ortalama Kelime Uzunlukları': word_lengths,
        'Ortalama Kelime Tekrarı ':repeated_word,
        'Duygu Analizi':emtion_analysis}
pop_df = pd.DataFrame(data)
pop_df["eski_turkce_kelime_orani"] = df_pop["eski_turkce_kelime_orani"].values
pop_df["fiil_orani"] = df_pop["fiil_orani"].values
pop_df["sifat_orani"] = df_pop["sifat_orani"].values
pop_df["zarf_orani"] = df_pop["zarf_orani"].values
pop_df["baglac_orani"] = df_pop["baglac_orani"].values
pop_df["unlem_orani"] = df_pop["unlem_orani"].values
pop_df["soru_orani"] = df_pop["soru_orani"].values
pop_df["bilinmeyen_orani"] = df_pop["bilinmeyen_orani"].values
pop_df['Label']=1

In [140]:
df_rock=df_dataset.loc[df_dataset["sarki_turu"]=='rock']

sarki_adi=[]
lengths=[]
word_lengths=[]
repeated_word=[]
emtion_analysis=[] 
rock_sarki_sozleri=[]
for index,row in df_rock.iterrows():
    sarki_adi.append(row['baslik'])
    rock_sarki_sozleri.append(row['sarki_sozu'])
    #row="".join(row)
   
    lyrics = row["sarki_sozu"]
    lengths.append(len(lyrics.split(" ")))
    temp=[]
    for word in lyrics.split(" "):
        temp.append(len(word))

    turkish_text = row["sarki_sozu"]
    blob = TextBlob(turkish_text)
    polarity = blob.sentiment.polarity

    if polarity > 0:
        if polarity >= 0.5:
           # print("Mutlu")
            emtion_analysis.append(5)
        else:
            #print("Şaşırmış")
            emtion_analysis.append(2)
    elif polarity < 0:
        if polarity <= -0.5:
            #print("Kızgın")
            emtion_analysis.append(3)
        else:
            #print("Üzgün")
            emtion_analysis.append(4)
    else:
        
        if "korku" in turkish_text.lower() or "korkmak" in turkish_text.lower():
            if polarity <= -0.3:
                #print("Korkmuş")
                emtion_analysis.append(1)
            else:
                #print("Nötr")
                emtion_analysis.append(0)
        else:
            #print("Nötr")
            emtion_analysis.append(0)
    word_lengths.append(mean(temp))

    counts = {}
    str_row=lyrics.split(("\n"))
    temp_List2=[]
    for sentence in str_row:
        for word in sentence.split(" "):
            #print(word)
            #print()

            if word not in counts:
                counts[word] = 0
            counts[word] += 1
    for item in counts.values():
        temp_List2.append(item)
        #print(item)
        #print()
    #sorted_c=sorted(counts)
    repeated_word.append(sum(temp_List2)/len(temp_List2))

rock_plot_data=[mean(lengths),mean(word_lengths),mean(repeated_word)]
data = {'Şarki Adı':sarki_adi,
        'Toplam Kelime Sayısı': lengths,
        'Ortalama Kelime Uzunlukları': word_lengths,
        'Ortalama Kelime Tekrarı ':repeated_word,
        'Duygu Analizi':emtion_analysis}
rock_df = pd.DataFrame(data)
rock_df["eski_turkce_kelime_orani"] = df_pop["eski_turkce_kelime_orani"].values
rock_df["fiil_orani"] = df_rock["fiil_orani"].values
rock_df["sifat_orani"] = df_rock["sifat_orani"].values
rock_df["zarf_orani"] = df_rock["zarf_orani"].values
rock_df["baglac_orani"] = df_rock["baglac_orani"].values
rock_df["unlem_orani"] = df_rock["unlem_orani"].values
rock_df["soru_orani"] = df_rock["soru_orani"].values
rock_df["bilinmeyen_orani"] = df_rock["bilinmeyen_orani"].values
rock_df['Label']=2

In [141]:
df_ilahi=df_dataset.loc[df_dataset["sarki_turu"]=='ilahi']

sarki_adi=[]
lengths=[]
word_lengths=[]
repeated_word=[]
emtion_analysis=[] 
ilahi_sarki_sozleri=[]
for index,row in df_ilahi.iterrows():
    sarki_adi.append(row['baslik'])
    ilahi_sarki_sozleri.append(row['sarki_sozu'])
    #row="".join(row)
   
    lyrics = row["sarki_sozu"]
    lengths.append(len(lyrics.split(" ")))
    temp=[]
    for word in lyrics.split(" "):
        temp.append(len(word))

    turkish_text = row["sarki_sozu"]
    blob = TextBlob(turkish_text)
    polarity = blob.sentiment.polarity

    if polarity > 0:
        if polarity >= 0.5:
           # print("Mutlu")
            emtion_analysis.append(5)
        else:
            #print("Şaşırmış")
            emtion_analysis.append(2)
    elif polarity < 0:
        if polarity <= -0.5:
            #print("Kızgın")
            emtion_analysis.append(3)
        else:
            #print("Üzgün")
            emtion_analysis.append(4)
    else:
        
        if "korku" in turkish_text.lower() or "korkmak" in turkish_text.lower():
            if polarity <= -0.3:
                #print("Korkmuş")
                emtion_analysis.append(1)
            else:
                #print("Nötr")
                emtion_analysis.append(0)
        else:
            #print("Nötr")
            emtion_analysis.append(0)
    word_lengths.append(mean(temp))

    counts = {}
    str_row=lyrics.split(("\n"))
    temp_List2=[]
    for sentence in str_row:
        for word in sentence.split(" "):
            #print(word)
            #print()

            if word not in counts:
                counts[word] = 0
            counts[word] += 1
    for item in counts.values():
        temp_List2.append(item)
        #print(item)
        #print()
    #sorted_c=sorted(counts)
    repeated_word.append(sum(temp_List2)/len(temp_List2))

ilahi_plot_data=[mean(lengths),mean(word_lengths),mean(repeated_word)]
data = {'Şarki Adı':sarki_adi,
        'Toplam Kelime Sayısı': lengths,
        'Ortalama Kelime Uzunlukları': word_lengths,
        'Ortalama Kelime Tekrarı ':repeated_word,
        'Duygu Analizi':emtion_analysis}
ilahi_df = pd.DataFrame(data)
ilahi_df["eski_turkce_kelime_orani"] = df_pop["eski_turkce_kelime_orani"].values
ilahi_df["fiil_orani"] = df_ilahi["fiil_orani"].values
ilahi_df["sifat_orani"] = df_ilahi["sifat_orani"].values
ilahi_df["zarf_orani"] = df_ilahi["zarf_orani"].values
ilahi_df["baglac_orani"] = df_ilahi["baglac_orani"].values
ilahi_df["unlem_orani"] = df_ilahi["unlem_orani"].values
ilahi_df["soru_orani"] = df_ilahi["soru_orani"].values
ilahi_df["bilinmeyen_orani"] = df_ilahi["bilinmeyen_orani"].values
ilahi_df['Label']=3

In [142]:
df_sanat=df_dataset.loc[df_dataset["sarki_turu"]=='sanat']

sarki_adi=[]
lengths=[]
word_lengths=[]
repeated_word=[]
emtion_analysis=[] 
sanat_sarki_sozleri=[]
for index,row in df_sanat.iterrows():
    sarki_adi.append(row['baslik'])
    sanat_sarki_sozleri.append(row['sarki_sozu'])
    #row="".join(row)
   
    lyrics = row["sarki_sozu"]
    lengths.append(len(lyrics.split(" ")))
    temp=[]
    for word in lyrics.split(" "):
        temp.append(len(word))

    turkish_text = row["sarki_sozu"]
    blob = TextBlob(turkish_text)
    polarity = blob.sentiment.polarity

    if polarity > 0:
        if polarity >= 0.5:
           # print("Mutlu")
            emtion_analysis.append(5)
        else:
            #print("Şaşırmış")
            emtion_analysis.append(2)
    elif polarity < 0:
        if polarity <= -0.5:
            #print("Kızgın")
            emtion_analysis.append(3)
        else:
            #print("Üzgün")
            emtion_analysis.append(4)
    else:
        
        if "korku" in turkish_text.lower() or "korkmak" in turkish_text.lower():
            if polarity <= -0.3:
                #print("Korkmuş")
                emtion_analysis.append(1)
            else:
                #print("Nötr")
                emtion_analysis.append(0)
        else:
            #print("Nötr")
            emtion_analysis.append(0)
    word_lengths.append(mean(temp))

    counts = {}
    str_row=lyrics.split(("\n"))
    temp_List2=[]
    for sentence in str_row:
        for word in sentence.split(" "):
            #print(word)
            #print()

            if word not in counts:
                counts[word] = 0
            counts[word] += 1
    for item in counts.values():
        temp_List2.append(item)
        #print(item)
        #print()
    #sorted_c=sorted(counts)
    repeated_word.append(sum(temp_List2)/len(temp_List2))

sanat_plot_data=[mean(lengths),mean(word_lengths),mean(repeated_word)]
data = {'Şarki Adı':sarki_adi,
        'Toplam Kelime Sayısı': lengths,
        'Ortalama Kelime Uzunlukları': word_lengths,
        'Ortalama Kelime Tekrarı ':repeated_word,
        'Duygu Analizi':emtion_analysis}
sanat_df = pd.DataFrame(data)
sanat_df["eski_turkce_kelime_orani"] = df_pop["eski_turkce_kelime_orani"].values
sanat_df["fiil_orani"] = df_sanat["fiil_orani"].values
sanat_df["sifat_orani"] = df_sanat["sifat_orani"].values
sanat_df["zarf_orani"] = df_sanat["zarf_orani"].values
sanat_df["baglac_orani"] = df_sanat["baglac_orani"].values
sanat_df["unlem_orani"] = df_sanat["unlem_orani"].values
sanat_df["soru_orani"] = df_sanat["soru_orani"].values
sanat_df["bilinmeyen_orani"] = df_sanat["bilinmeyen_orani"].values
sanat_df['Label']=4

In [143]:
df_halk=df_dataset.loc[df_dataset["sarki_turu"]=='halk']

sarki_adi=[]
lengths=[]
word_lengths=[]
repeated_word=[]
emtion_analysis=[] 
halk_sarki_sozleri=[]
for index,row in df_halk.iterrows():
    sarki_adi.append(row['baslik'])
    halk_sarki_sozleri.append(row['sarki_sozu'])
    #row="".join(row)
   
    lyrics = row["sarki_sozu"]
    lengths.append(len(lyrics.split(" ")))
    temp=[]
    for word in lyrics.split(" "):
        temp.append(len(word))

    turkish_text = row["sarki_sozu"]
    blob = TextBlob(turkish_text)
    polarity = blob.sentiment.polarity

    if polarity > 0:
        if polarity >= 0.5:
           # print("Mutlu")
            emtion_analysis.append(5)
        else:
            #print("Şaşırmış")
            emtion_analysis.append(2)
    elif polarity < 0:
        if polarity <= -0.5:
            #print("Kızgın")
            emtion_analysis.append(3)
        else:
            #print("Üzgün")
            emtion_analysis.append(4)
    else:
        
        if "korku" in turkish_text.lower() or "korkmak" in turkish_text.lower():
            if polarity <= -0.3:
                #print("Korkmuş")
                emtion_analysis.append(1)
            else:
                #print("Nötr")
                emtion_analysis.append(0)
        else:
            #print("Nötr")
            emtion_analysis.append(0)
    word_lengths.append(mean(temp))

    counts = {}
    str_row=lyrics.split(("\n"))
    temp_List2=[]
    for sentence in str_row:
        for word in sentence.split(" "):
            #print(word)
            #print()

            if word not in counts:
                counts[word] = 0
            counts[word] += 1
    for item in counts.values():
        temp_List2.append(item)
        #print(item)
        #print()
    #sorted_c=sorted(counts)
    repeated_word.append(sum(temp_List2)/len(temp_List2))

halk_plot_data=[mean(lengths),mean(word_lengths),mean(repeated_word)]
data = {'Şarki Adı':sarki_adi,
        'Toplam Kelime Sayısı': lengths,
        'Ortalama Kelime Uzunlukları': word_lengths,
        'Ortalama Kelime Tekrarı ':repeated_word,
        'Duygu Analizi':emtion_analysis}
halk_df = pd.DataFrame(data)
halk_df["eski_turkce_kelime_orani"] = df_pop["eski_turkce_kelime_orani"].values
halk_df["fiil_orani"] = df_halk["fiil_orani"].values
halk_df["sifat_orani"] = df_halk["sifat_orani"].values
halk_df["zarf_orani"] = df_halk["zarf_orani"].values
halk_df["baglac_orani"] = df_halk["baglac_orani"].values
halk_df["unlem_orani"] = df_halk["unlem_orani"].values
halk_df["soru_orani"] = df_halk["soru_orani"].values
halk_df["bilinmeyen_orani"] = df_halk["bilinmeyen_orani"].values
halk_df['Label']=5

# Base Model Training

In [144]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

# Dataset 1
data1 = np.array(rap_sarki_sozleri)
rep_label = np.ones((200,))*0


# Dataset 2
data2 = np.array(pop_sarki_sozleri)
pop_label = np.ones((200,))

# Dataset 3
data3 = np.array(rock_sarki_sozleri)
rock_label = np.ones((200,))*2

# Dataset 4
data4 = np.array(ilahi_sarki_sozleri)
ilahi_label = np.ones((200,))*3

# Dataset 5
data5 = np.array(sanat_sarki_sozleri)
sanat_label = np.ones((200,))*4

# Dataset 6
data6 = np.array(halk_sarki_sozleri)
halk_label = np.ones((200,))*5

X = np.concatenate((data1, data2, data3,data4, data5, data6))
y = np.concatenate((rep_label, pop_label, rock_label,ilahi_label, sanat_label, halk_label))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=38)

vectorizer = CountVectorizer(ngram_range=(2,2))

X = vectorizer.fit_transform(X_train)

svm = SVC(kernel='linear', random_state=0)
svm.fit(X, y_train)

SVC(kernel='linear', random_state=0)

# Base Model Test and Results

In [145]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

X_test_after_NGram = vectorizer.transform(X_test)
y_pred = svm.predict(X_test_after_NGram)

print("Predictions:", y_test)
print("reel:", y_pred)
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred, average='weighted')
recall = recall_score(y_test, y_pred, average='weighted')
f1 = f1_score(y_test, y_pred, average='weighted')
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 score:", f1)

Predictions: [0. 4. 5. 2. 1. 2. 1. 2. 1. 0. 2. 0. 0. 0. 0. 3. 0. 1. 2. 0. 5. 2. 4. 5.
 5. 3. 5. 3. 4. 5. 0. 5. 1. 1. 5. 5. 2. 0. 5. 3. 3. 1. 3. 2. 5. 4. 4. 2.
 2. 2. 5. 4. 4. 3. 3. 3. 4. 0. 4. 5. 0. 2. 1. 1. 3. 4. 3. 2. 5. 0. 3. 2.
 0. 5. 3. 5. 5. 1. 3. 1. 3. 0. 2. 0. 0. 4. 4. 0. 4. 5. 4. 4. 0. 2. 4. 0.
 3. 5. 4. 4. 5. 3. 2. 1. 4. 0. 0. 1. 0. 2. 4. 0. 5. 2. 3. 5. 4. 2. 4. 5.]
reel: [4. 4. 4. 4. 4. 2. 4. 2. 4. 4. 1. 4. 4. 0. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.
 4. 3. 4. 4. 5. 4. 1. 4. 4. 4. 4. 4. 4. 1. 4. 4. 4. 4. 1. 4. 4. 4. 4. 4.
 2. 1. 4. 4. 4. 4. 4. 4. 4. 1. 4. 5. 4. 2. 4. 4. 4. 4. 4. 2. 4. 4. 3. 1.
 1. 4. 4. 4. 4. 4. 4. 4. 4. 1. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4. 5. 4. 4.
 4. 4. 4. 4. 4. 3. 2. 4. 4. 4. 4. 4. 4. 1. 4. 4. 4. 4. 4. 4. 4. 4. 4. 4.]
Accuracy: 0.26666666666666666
Precision: 0.6202462772050402
Recall: 0.26666666666666666
F1 score: 0.21522969187675073


# Makine Öğrenmesi Modelleri

In [146]:
# Dataset 1
data1 = np.array(rap_sarki_sozleri)
rep_label = np.ones((200,))*0


# Dataset 2
data2 = np.array(pop_sarki_sozleri)
pop_label = np.ones((200,))

# Dataset 3
data3 = np.array(rock_sarki_sozleri)
rock_label = np.ones((200,))*2

# Dataset 4
data4 = np.array(ilahi_sarki_sozleri)
ilahi_label = np.ones((200,))*3

# Dataset 5
data5 = np.array(sanat_sarki_sozleri)
sanat_label = np.ones((200,))*4

# Dataset 6
data6 = np.array(halk_sarki_sozleri)
halk_label = np.ones((200,))*5
rap_df['sarki_sozu']=data1
pop_df['sarki_sozu']=data2
rock_df['sarki_sozu']=data3
ilahi_df['sarki_sozu']=data4
sanat_df['sarki_sozu']=data5
halk_df['sarki_sozu']=data6

# Random Forest Without Emotion Analysis

In [147]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score


datasets = [rap_df, pop_df, rock_df, ilahi_df, sanat_df, halk_df]
data = pd.concat(datasets, ignore_index=True,axis=0)

text_data = data['sarki_sozu']

vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using both unigrams and bigrams
text_features = vectorizer.fit_transform(text_data)

text_features_df = pd.DataFrame(text_features.toarray(), columns=vectorizer.get_feature_names())

# Combine the n-gram features with the original features
X = data.drop(['sarki_sozu','Şarki Adı','Duygu Analizi','Label'], axis=1)  # Replace 'label' with the name of your label column
X = pd.concat([X, text_features_df], axis=1)

scaler = StandardScaler()
X = scaler.fit_transform(X)

y = data['Label']  # Replace 'label' with the name of your label column

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

best_rf_clf = RandomForestClassifier(max_depth= 30, min_samples_leaf= 1, min_samples_split= 5, 
                                     n_estimators=600, random_state=42)
best_rf_clf.fit(X_train, y_train)

y_pred = best_rf_clf.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)



c:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Accuracy: 0.75
Classification Report:
               precision    recall  f1-score   support

           0       0.89      1.00      0.94        39
           1       0.68      0.56      0.61        48
           2       0.80      0.57      0.67        35
           3       0.93      0.74      0.82        38
           4       0.66      0.73      0.69        45
           5       0.65      0.94      0.77        35

    accuracy                           0.75       240
   macro avg       0.77      0.76      0.75       240
weighted avg       0.76      0.75      0.75       240



# RANDOM FOREST CLASİFİER

GridSearchCV Kullanılarak parametreler optimize edilmiştir.

rf_clf = RandomForestClassifier(n_estimators=100, random_state=42)
param_grid = {
    'n_estimators': [10, 50, 100, 200,300,400,500,600],
    'max_depth': [None, 10, 20, 30,40,50],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}


grid_search = GridSearchCV(rf_clf, param_grid, cv=1, verbose=2, n_jobs=-1)


grid_search.fit(X_train, y_train)

Sonucunda

Best Parameters: {'max_depth': 30, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 600}

Çıkmıştır

In [148]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# Assuming your data is in a CSV file called 'data.csv'
datasets = [rap_df, pop_df, rock_df, ilahi_df, sanat_df, halk_df]
data = pd.concat(datasets, ignore_index=True,axis=0)

text_data = data['sarki_sozu']

# Extract n-grams from the text data
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using both unigrams and bigrams
text_features = vectorizer.fit_transform(text_data)

# Convert n-grams features to a DataFrame
text_features_df = pd.DataFrame(text_features.toarray(), columns=vectorizer.get_feature_names())

# Combine the n-gram features with the original features
X = data.drop(['sarki_sozu','Şarki Adı','Label'], axis=1)  # Replace 'label' with the name of your label column
X = pd.concat([X, text_features_df], axis=1)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Separate the label
y = data['Label']  # Replace 'label' with the name of your label column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the RandomForest classifier with the best parameters
best_rf_clf = RandomForestClassifier(max_depth= 30, min_samples_leaf= 1, min_samples_split= 5, 
                                     n_estimators=600, random_state=42)
best_rf_clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = best_rf_clf.predict(X_test)

# Calculate and print the accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

c:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Accuracy: 0.77
Classification Report:
               precision    recall  f1-score   support

           0       0.93      1.00      0.96        39
           1       0.78      0.58      0.67        48
           2       0.78      0.60      0.68        35
           3       0.97      0.76      0.85        38
           4       0.63      0.80      0.71        45
           5       0.65      0.89      0.75        35

    accuracy                           0.77       240
   macro avg       0.79      0.77      0.77       240
weighted avg       0.79      0.77      0.76       240



Duygu analizi özniteliği çıkarıldığı durumda model daha kötü performans sergilemiştir.

# SVM CLASİFİER

In [149]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Assuming your data is in a CSV file called 'data.csv'
datasets = [rap_df, pop_df, rock_df, ilahi_df, sanat_df, halk_df]
data = pd.concat(datasets, ignore_index=True,axis=0)

text_data = data['sarki_sozu']

# Extract n-grams from the text data
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using both unigrams and bigrams
text_features = vectorizer.fit_transform(text_data)

# Convert n-grams features to a DataFrame
text_features_df = pd.DataFrame(text_features.toarray(), columns=vectorizer.get_feature_names())

# Combine the n-gram features with the original features
X = data.drop(['sarki_sozu','Şarki Adı','Label'], axis=1)  # Replace 'label' with the name of your label column
X = pd.concat([X, text_features_df], axis=1)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Separate the label
y = data['Label']  # Replace 'label' with the name of your label column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Train the RandomForest classifier with the best parameters


best_svm_clf = SVC(random_state=0)

best_svm_clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = best_svm_clf.predict(X_test)

# Calculate and print the accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)

c:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Accuracy: 0.56
Classification Report:
               precision    recall  f1-score   support

           0       0.84      0.96      0.90        28
           1       0.83      0.21      0.33        24
           2       1.00      0.22      0.36        18
           3       0.54      0.50      0.52        14
           4       0.42      0.58      0.49        19
           5       0.33      0.76      0.46        17

    accuracy                           0.56       120
   macro avg       0.66      0.54      0.51       120
weighted avg       0.69      0.56      0.53       120



# Naive Bayes

In [150]:
from sklearn.naive_bayes import GaussianNB  
datasets = [rap_df, pop_df, rock_df, ilahi_df, sanat_df, halk_df]
data = pd.concat(datasets, ignore_index=True,axis=0)

text_data = data['sarki_sozu']

# Extract n-grams from the text data
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using both unigrams and bigrams
text_features = vectorizer.fit_transform(text_data)

# Convert n-grams features to a DataFrame
text_features_df = pd.DataFrame(text_features.toarray(), columns=vectorizer.get_feature_names())

# Combine the n-gram features with the original features
X = data.drop(['sarki_sozu','Şarki Adı','Label'], axis=1)  # Replace 'label' with the name of your label column
X = pd.concat([X, text_features_df], axis=1)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Separate the label
y = data['Label']  # Replace 'label' with the name of your label column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

# Train the RandomForest classifier with the best parameters

# Fitting Naive Bayes to the Training set  

best_NB_clf = GaussianNB()  
best_NB_clf.fit(X_train, y_train)  


# Make predictions on the testing set
y_pred = best_NB_clf.predict(X_test)

# Calculate and print the accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


c:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Accuracy: 0.54
Classification Report:
               precision    recall  f1-score   support

           0       0.43      0.96      0.59        28
           1       0.42      0.21      0.28        24
           2       0.62      0.28      0.38        18
           3       0.81      0.93      0.87        14
           4       0.67      0.11      0.18        19
           5       0.72      0.76      0.74        17

    accuracy                           0.54       120
   macro avg       0.61      0.54      0.51       120
weighted avg       0.58      0.54      0.49       120



# XGBoost

In [151]:
import xgboost as xgb
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split

# Assuming your data is in a CSV file called 'data.csv'
datasets = [rap_df, pop_df, rock_df, ilahi_df, sanat_df, halk_df]
data = pd.concat(datasets, ignore_index=True,axis=0)

text_data = data['sarki_sozu']

# Extract n-grams from the text data
vectorizer = CountVectorizer(ngram_range=(1, 2))  # Using both unigrams and bigrams
text_features = vectorizer.fit_transform(text_data)

# Convert n-grams features to a DataFrame
text_features_df = pd.DataFrame(text_features.toarray(), columns=vectorizer.get_feature_names())

# Combine the n-gram features with the original features
X = data.drop(['sarki_sozu','Şarki Adı','Label'], axis=1)  # Replace 'label' with the name of your label column
X = pd.concat([X, text_features_df], axis=1)

# Standardize the features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Separate the label
y = data['Label']  # Replace 'label' with the name of your label column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

best_xgb_clf = xgb.XGBClassifier(max_depth=3, learning_rate=0.1, n_estimators=100)

# Train the model on the training data
best_xgb_clf.fit(X_train, y_train)

# Make predictions on the testing set
y_pred = best_xgb_clf.predict(X_test)

# Calculate and print the accuracy and classification report
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

report = classification_report(y_test, y_pred)
print("Classification Report:\n", report)


c:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


Accuracy: 0.70
Classification Report:
               precision    recall  f1-score   support

           0       0.96      0.96      0.96        28
           1       0.67      0.50      0.57        24
           2       0.50      0.56      0.53        18
           3       0.77      0.71      0.74        14
           4       0.56      0.53      0.54        19
           5       0.65      0.88      0.75        17

    accuracy                           0.70       120
   macro avg       0.68      0.69      0.68       120
weighted avg       0.70      0.70      0.70       120

